In [ ]:
import yaml
import wandb
import time
import os
import pandas as pd



In [ ]:
with open('config/config.yaml') as f:
    default_config = yaml.load(f, Loader=yaml.FullLoader)
#display(default_config)

In [40]:
import easydict

args = easydict.EasyDict(default_config)
print(type(args.WEIGHT_DECAY))
print(args.WEIGHT_DECAY)
args.WEIGHT_DECAY = float(args.WEIGHT_DECAY)
print(type(args.WEIGHT_DECAY))
print(args.WEIGHT_DECAY)


<class 'str'>
1e-06
<class 'float'>
1e-06


모델이름은 직접 할당해줘야됨!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
################# 돌릴 모델 바꾸려면 여기서 dlfma 바꿔줘야함
args.MODEL = "DeepCoNN"
print(args.DEEPCONN_VECTOR_CREATE)
print(type(args.DEEPCONN_VECTOR_CREATE))

MAIN.PY  모델 도입부

In [ ]:
from src import (CNN_FM, DeepCoNN, DeepCrossNetworkModel,
                 FactorizationMachineModel,
                 FieldAwareFactorizationMachineModel,
                 NeuralCollaborativeFiltering, WideAndDeepModel,
                 seed_everything)
from src.data import (context_data_load, context_data_loader,
                      context_data_split, dl_data_load, dl_data_loader,
                      dl_data_split, image_data_load, image_data_loader,
                      image_data_split, text_data_load, text_data_loader,
                      text_data_split)

In [ ]:
######################## DATA LOAD
print(f'--------------- {args.MODEL} Load Data ---------------')
if args.MODEL in ('FM', 'FFM'):
    data = context_data_load(args)
elif args.MODEL in ('NCF', 'WDN', 'DCN'):
    data = dl_data_load(args)
elif args.MODEL == 'CNN_FM':
    data = image_data_load(args)
elif args.MODEL == 'DeepCoNN':
    import nltk
    nltk.download('punkt')
    data = text_data_load(args)
else:
    pass

######################## Train/Valid Split
print(f'--------------- {args.MODEL} Train/Valid Split ---------------')
if args.MODEL in ('FM', 'FFM'):
    data = context_data_split(args, data)
    data = context_data_loader(args, data)
elif args.MODEL in ('NCF', 'WDN', 'DCN'):
    data = dl_data_split(args, data)
    data = dl_data_loader(args, data)
elif args.MODEL=='CNN_FM':
    data = image_data_split(args, data)
    data = image_data_loader(args, data)
elif args.MODEL=='DeepCoNN':
    data = text_data_split(args, data)
    data = text_data_loader(args, data)
else:
    pass
data

In [ ]:
class sweep:
    def __init__(self, args, data,config):
        super().__init__()
        self.args = args
        self.data = data
        self.config = config

        os.environ["WANDB_NOTEBOOK_NAME"] = "/opt/ml/input/level1_bookratingprediction_recsys-level1-recsys-05/sweep.ipynb"
        


    def train(self):
        wandb.init(project="sweep_DeepCoNN",
            config=self.config,
            settings=dict(start_method='thread')
        )    
        data = self.data
        args = self.args
        wandb.log({"emb_dim": wandb.config.emb_dim,
        "latent_dim":wandb.config.latent_dim})
        print(f'--------------- INIT {args.MODEL} ---------------')
        if args.MODEL=='FM':
            print("wandb.config.emb_dim = ", wandb.config.emb_dim)
            print(type(wandb.config.emb_dim))
            args.FM_EMBED_DIM = wandb.config.emb_dim
            model = FactorizationMachineModel(args, data)
        elif args.MODEL=='FFM':
            args.FFM_EMBED_DIM = wandb.config.emb_dim
            model = FieldAwareFactorizationMachineModel(args, data)
        elif args.MODEL=='NCF':
            tmp = (wandb.config.mlp_dim,wandb.config.mlp_dim)
            print("wandb.config.emb_dim = ", wandb.config.emb_dim)
            print("wandb.config.mlp_dim = ", tmp)
            print("wandb.config.drop_out = ", wandb.config.drop_out)
            args.NCF_EMBED_DIM = wandb.config.emb_dim
            args.NCF_MLP_DIMS = tmp
            args.NCF_DROPOUT = wandb.config.drop_out
            model = NeuralCollaborativeFiltering(args, data)
        elif args.MODEL=='WDN':
            args.WDN_EMBED_DIM = wandb.config.emb_dim
            model = WideAndDeepModel(args, data)
        elif args.MODEL=='DCN':
            args.DCN_EMBED_DIM = wandb.config.emb_dim
            model = DeepCrossNetworkModel(args, data)
        elif args.MODEL=='CNN_FM':
            args.CNN_FM_EMBED_DIM = wandb.config.emb_dim
            model = CNN_FM(args, data)
        elif args.MODEL=='DeepCoNN':
            print("wandb.config.emb_dim = ", wandb.config.emb_dim)
            print("wandb.config.latent_dim = ", wandb.config.latent_dim)
            args.DEEPCONN_EMBED_DIM = wandb.config.emb_dim
            args.DEEPCONN_LATENT_DIM = wandb.config.latent_dim
            model = DeepCoNN(args, data)
        else:
            pass

        #wandb.config.update(args)
        # wandb.watch(model)
        ######################## TRAIN
        print(f'--------------- {args.MODEL} TRAINING ---------------')
        rmse = model.train()
        ######################## INFERENCE
        print(f'--------------- {args.MODEL} PREDICT ---------------')
        if args.MODEL in ('FM', 'FFM', 'NCF', 'WDN', 'DCN'):
            predicts = model.predict(data['test_dataloader'])
        elif args.MODEL=='CNN_FM':
            predicts  = model.predict(data['test_dataloader'])
        elif args.MODEL=='DeepCoNN':
            predicts  = model.predict(data['test_dataloader'])
        else:
            pass
        
        ######################## PREDICT GET IN RANGE
        def adjust_predict(y):
            if y < 1.0:
                return 1.0
            elif y > 10.0:
                return 10.0
            return y
        
        predicts = list(map(adjust_predict, predicts))

        ######################## SAVE PREDICT
        print(f'--------------- SAVE {args.MODEL} PREDICT ---------------')
        submission = pd.read_csv(args.DATA_PATH + 'sample_submission.csv')
        if args.MODEL in ('FM', 'FFM', 'NCF', 'WDN', 'DCN', 'CNN_FM', 'DeepCoNN'):
            submission['rating'] = predicts
        else:
            pass
        train1 = pd.read_csv('./data/train_ratings.csv')

        count=train1.groupby("user_id").size()
        dfcount = pd.DataFrame(count, columns=["count"])
        submission=pd.merge(submission,dfcount, how='left', on='user_id')
        submission['count'] = submission['count'].fillna(0)
        submission.set_index("user_id",inplace = True)

        for row in submission.itertuples():
            if row[3] == 0 :
                submission.at[row[0],"rating"] = 7

        submission = submission.reset_index()
        submission = submission.drop(['count'], axis=1)


        now = time.localtime()
        now_date = time.strftime('%Y%m%d', now)
        now_hour = time.strftime('%X', now)
        save_time = now_date + '_' + now_hour.replace(':', '')
        submission.to_csv('submit/{}_{}_{}.csv'.format(save_time, args.MODEL, round(rmse, 5), index=False))

        wandb.finish()
    

여기 아래에 있는 sweep_configuration 에 있는 Parameters 값에 있는 것들만 랜덤 생성됨.

지금은 FM, FFM에 있는 embeding dim을 실험하기 위한 코드
나머지 모델들 돌릴경우에는 아래에 추가해주면 됨.

처음에 
################# 돌릴 모델 바꾸려면 여기서
args.MODEL = "FM"

코드에서 돌릴 모델 이름 써줘야 됨~!!!!!

In [ ]:
"""
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'rmse'}, 
    'parameters':{
        'emb_dim':{'values': [16,12,8,4,2]},
        'mlp_dim':{'values': [16,12,8,4,2]},
        'drop_out':{'values': [0.2,0.05]}
        }}
config={
        "epochs": args.EPOCHS,
        "batch_size": args.BATCH_SIZE,
        "lr": args.LR,
        "emb_dim": 16,
        'mlp_dim': (16,16),
        'drop_out': 0.2
        }
"""
######이거는 NCF


In [ ]:
########DeepCoNN
config={
        "epochs": args.EPOCHS,
        "batch_size": args.BATCH_SIZE,
        "lr": args.LR,
        "emb_dim": 32,
        "latent_dim":10,
        "conv1doutdim":50,
        "kernel_size":3,
        "word_dim":768,
        "out_dim":32
        }
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_loss'}, 
    'parameters':{
        "emb_dim": {'max':64, 'min':2},
        "latent_dim":{'max':20, 'min':2}
        }}


In [ ]:
wandb.login()

In [ ]:
#args.EPOCHS = 1

In [ ]:
%env "WANDB_NOTEBOOK_NAME" "/opt/ml/input/level1_bookratingprediction_recsys-level1-recsys-05/sweep.ipynb"


In [ ]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "/opt/ml/input/level1_bookratingprediction_recsys-level1-recsys-05/sweep.ipynb"

In [ ]:
sibal = sweep(args, data,config)


wandb.agent(sweep_id, function=sibal.train,count =3)

count = ""
여기에 몇번 돌릴지 넣으면 돌아감!!!!


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration,project='sweep_DeepCoNN')
wandb.agent(sweep_id, function=sibal.train,count =100)